# Scott Breitbach
## Milestone 5: Merging the Data and Storing in a Database/Visualizing Data
## 26-Feb-2021
## DSC540, Weeks 11-12

In [1]:
import sqlite3
import pandas as pd

# Load data into an SQL database, each as an individual table.

In [2]:
# Load the previously cleaned data from CSV files:
csvData = pd.read_csv("CSV_nebraskaCleaned.csv")
webData = pd.read_csv("WEB_NEbeerReddit.csv")
apiData = pd.read_csv("API_beerMapping.csv")

## First, a little extra cleaning

#### Replace spaces in column names with underscores:

In [3]:
def colSpaceToUnderscore(df):
    '''Replaces spaces with underscores in the column names of a DataFrame'''
    newCols = []
    for c in df.columns:
        newCols.append(c.replace(' ', '_'))
    df.columns = newCols
    print(f"New column names:\n{df.columns}")  

In [4]:
colSpaceToUnderscore(csvData)
colSpaceToUnderscore(webData)
colSpaceToUnderscore(apiData)

New column names:
Index(['Brewery_ID', 'Brewery_Name', 'Type_of_Brewery', 'Street_Address',
       'City', 'State', 'Postal_Code', '5-digit_Zip', 'Website', 'Phone',
       'Longitude', 'Latitude'],
      dtype='object')
New column names:
Index(['Brewery_Name', 'City', 'State', 'Type_of_Brewery',
       'Type_of_Distribution', 'Notes', '#_Locations', 'Locations'],
      dtype='object')
New column names:
Index(['BreweryID', 'BreweryName', 'Type', 'ReviewLink', 'ProxyLink', 'Map',
       'StreetAddress', 'City', 'State', 'Zip', 'Country', 'PhoneNum',
       'Website', 'Rating', 'ImageCount'],
      dtype='object')


#### Make street addresses a uniform format:

In [5]:
import pandas_usaddress

def standardizeAddress(df, colName):
    '''Creates street address uniformity and makes Title Case;
    Replaces existing street address column.'''
    tempDF = pd.DataFrame()
    addressDF = pd.DataFrame()
    
    tempDF = pandas_usaddress.tag(df, [colName], granularity='single', standardize=True) 
    addressDF['Address'] = tempDF.SingleLine.str.title()
    tempDF = pandas_usaddress.tag(df, [colName], granularity='medium', standardize=True) 
    addressDF['Suffix'] = tempDF.OccupancySuite.str.title()
    
    for i in range(len(addressDF.Address)):
        addressDF['Address'][i] = addressDF.Address[i].rstrip(str(addressDF.Suffix[i])).strip()
        
    df[colName] = addressDF['Address']

In [6]:
standardizeAddress(csvData, 'Street_Address')
standardizeAddress(apiData, 'StreetAddress')

In [78]:
# A few manual fixes:
csvData['Street_Address'] = csvData['Street_Address'].str.replace(' Blackstone District', '')
csvData['Street_Address'] = csvData['Street_Address'].str.replace('500 W South', '500 W South St')
apiData['StreetAddress'] = apiData['StreetAddress'].str.replace('710Th Rd', '62193 710Th Rd')
csvData.State = 'NE'
webData.State = 'NE'

#### Add a column for consolidating Omaha suburbs:
https://en.wikipedia.org/wiki/Neighborhoods_of_Omaha,_Nebraska

In [7]:
import requests
# Use table from Wikipedia page to generage a list of suburbs:
url = "https://en.wikipedia.org/wiki/Neighborhoods_of_Omaha,_Nebraska"
listOfTables = pd.read_html(url)
omahaSubsDF = listOfTables[1]
omahaSubs = list(omahaSubsDF['Neighborhoods in Omaha alphabetical order'][1:])

In [8]:
def fixCitySpacing(df):
    df['City'].replace('Mc Cook', 'McCook', inplace=True)
    df['City'].replace('LaVista', 'La Vista', inplace=True)

In [9]:
fixCitySpacing(csvData)
fixCitySpacing(webData)
fixCitySpacing(apiData)

In [10]:
def omahaSuburbs(df, colName):
    '''Creates a copy of the City column and converts any locations 
    found in the list of Omaha suburbs to "Omaha"'''
    df['Central_City'] = df[colName]
    df.loc[(df['Central_City'].isin(omahaSubs) == True), 'Central_City'] = "Omaha"

In [11]:
omahaSuburbs(csvData, 'City')
omahaSuburbs(webData, 'City')
omahaSuburbs(apiData, 'City')

#### Standardize phone numbers:

In [12]:
import phonenumbers as pn
import numpy as np

def fixPhoneNums(df, colName):
    '''Attempts to convert each phone number to a standardized format and add
    to a list, adding NaN if it fails. Assigns the list back to the DF column.'''
    phList = []
    for i in range(len(df[colName])):
        try:
            phList.append(pn.format_number(pn.parse(df[colName][i], 'US'), 
                                           pn.PhoneNumberFormat.NATIONAL))
        except:
            phList.append(np.nan)
    df[colName] = phList

In [13]:
fixPhoneNums(apiData, 'PhoneNum')

#### Standardize Brewery Names:

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzyMatch(df, colName, scorer=fuzz.ratio, display=False):
    '''Fuzzy matches a column of brewery names against a standard set of names.
    `display=True`  returns a DF comparing old with new names plus a score.
    `display=False` returns a Series of the corrected names.'''
    checkDF = df.copy()
    actualDF = webData.copy()
    testDF = pd.DataFrame()
    testDF['NamesToCheck'] = checkDF[colName]
    
    actualName = []
    similarity = []
    for i in testDF['NamesToCheck']:
        ratio = process.extract(i, actualDF['Brewery_Name'], 
                                limit=1, scorer=scorer) 
        actualName.append(ratio[0][0])
        similarity.append(ratio[0][1])
        testDF['NameMatch'] = pd.Series(actualName)
        testDF['Score'] = pd.Series(similarity)
    
    if display == True:
        print('Average Score: {0:.2f}'.format(testDF['Score'].mean()))
        return testDF.sort_values('Score', ascending=False)
    elif display == False:
        return testDF['NameMatch']

In [15]:
csvData['Brewery_Name'] = fuzzyMatch(csvData, 'Brewery_Name', 
                                     fuzz.partial_ratio, display=False)

In [16]:
apiData['BreweryName'] = fuzzyMatch(apiData, 'BreweryName', 
                                    fuzz.token_sort_ratio, display=False)

#### Remove Granite City (not a Nebraska brewery):

In [17]:
def removeGC(df, colName):
    df = df[df[colName] != 'Granite City Food & Brewery']

In [18]:
removeGC(csvData, 'Brewery_Name')
removeGC(webData, 'Brewery_Name')
removeGC(apiData, 'BreweryName')

## Load DataFrames into an SQL Database

In [19]:
# Load data into an SQL database, each as an individual table:
with sqlite3.connect('Nebeerska.db') as conn:
    cursor = conn.cursor()
    csvData.to_sql('csvtable', conn, if_exists='replace', index=False)
    webData.to_sql('webtable', conn, if_exists='replace', index=False)
    apiData.to_sql('apitable', conn, if_exists='replace', index=False)

### Get a list of the tables:

In [20]:
res = conn.execute('''SELECT name FROM sqlite_master WHERE type='table';''')
tableList = []
for name in res:
    print(name[0])
    tableList.append(name[0])

csvtable
webtable
apitable


### Check out the first few rows of each table:

In [21]:
def head(table, nRows=5):
    '''Prints the first n rows of the table specified'''
    cursor.execute(f'''SELECT * FROM {table}''')
    for row in cursor.fetchall()[:nRows]:
        print(row)

In [22]:
for i in tableList:
    print(f"Table: '{i}' - ")
    head(i, 3)
    print()

Table: 'csvtable' - 
('backswing-brewing-co-lincoln', 'Backswing Brewing Co.', 'Micro', '500 W South', 'Lincoln', 'Nebraska', '68522-1744', 68522, 'http://www.backswingbrewing.com', '(402) 515-4263', None, None, 'Lincoln')
('benson-brewery-omaha', 'Benson Brewery', 'Brewpub', '6059 Maple St', 'Omaha', 'Nebraska', '68104-4050', 68104, 'http://www.bensonbrewery.com', '(402) 934-8668', -96.00634511, 41.28489756, 'Omaha')
('boiler-brewing-company-lincoln', 'Boiler Brewing Company', 'Micro', '129 N 10Th', 'Lincoln', 'Nebraska', '68508-3633', 68508, 'http://www.boilerbrewingcompany.com', '(402) 261-8775', None, None, 'Lincoln')

Table: 'webtable' - 
('Backswing Brewing Co.', 'Lincoln', 'Nebraska', 'Micro', 'On-tap at the brewery and at select locations', 'Distro planned for 2016', 1, 'Lincoln', 'Lincoln')
('Benson Brewery', 'Omaha', 'Nebraska', 'Taproom', 'On tap only at the brewery', 'Other beers, locals included, on tap. Omaha Brewing is the parent company', 1, 'Omaha', 'Omaha')
('Boiler B

# Join the datasets together into 1 dataset.

# Create 5 visualizations that demonstrate the data.
Note: At least 2 should have data from > 1 source.